In [1]:
import tweepy

In [ ]:
'https://twitter.com/alvarodias_'
'https://twitter.com/cirogomes'
'https://twitter.com/guilhermeboulos'
'https://twitter.com/jairbolsonaro'
'https://twitter.com/geraldoalckmin'
'https://twitter.com/marinasilva'
'https://twitter.com/lulaoficial'
'https://twitter.com/meirelles'

In [ ]:
consumer_key = 'J9GBoNaltM0QUJboFTMyUpONh'
consumer_secret = 'OjgOGkLf3vT4uDGggeu0mWq9xJcnpUqFUCdYIRQH02CkkLC0nr'

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)